<a href="https://colab.research.google.com/github/MarshallPotts/CSE450-Machine-Learning/blob/main/case_study4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dropout
from keras import Sequential
#!pip install tensorflow keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

print(tf.__version__)


2.18.0


In [28]:
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dteday        112475 non-null  object 
 1   hr            112475 non-null  float64
 2   casual        112475 non-null  int64  
 3   registered    112475 non-null  int64  
 4   temp_c        112475 non-null  float64
 5   feels_like_c  112475 non-null  float64
 6   hum           112475 non-null  float64
 7   windspeed     112475 non-null  float64
 8   weathersit    112475 non-null  int64  
 9   season        112475 non-null  int64  
 10  holiday       112475 non-null  int64  
 11  workingday    112475 non-null  int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 10.3+ MB


In [29]:
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [30]:
#drop NA values
bikes.dropna(axis=0, how="any", inplace=True)

In [31]:
#get the dteday into the right format to use for tensorflow
bikes_date_str = bikes['dteday'].astype(str)  #convert to a string
bikes_date = pd.to_datetime(bikes_date_str)   #change string to datetime
bikes["dteday"] = bikes_date  #add to frame
bikes["dteday"] = bikes["dteday"] + pd.to_timedelta(bikes["hr"], unit='h')  #add the hours as a datetime timedelta
bikes.tail()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
112470,2023-10-31 19:00:00,19.0,241,594,10.2,10.2,0.4516,8.4,2,4,0,1
112471,2023-10-31 20:00:00,20.0,171,450,8.9,7.5,0.5268,9.3,1,4,0,1
112472,2023-10-31 21:00:00,21.0,176,402,7.7,6.4,0.5756,7.8,1,4,0,1
112473,2023-10-31 22:00:00,22.0,106,257,8.0,8.0,0.5604,0.0,1,4,0,1
112474,2023-10-31 23:00:00,23.0,78,135,6.5,6.5,0.6401,4.5,1,4,0,1


In [32]:
#convert datetime into epochs
bikes['dteday'] = bikes['dteday'].astype(int) / 10**9 #convert to seconds

#remove hour column
bikes = bikes.drop(columns=['hr'])

bikes.head()

,dteday,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1.293840e+09,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1.293844e+09,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1.293847e+09,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1.293851e+09,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1.293854e+09,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [33]:
#create dataframe for the model
x = bikes.drop(columns=['registered','casual'])
y = bikes[['registered','casual']]
x.head()

,dteday,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1.293840e+09,3.0,3.0,0.7957,0.8,1,1,0,0
1,1.293844e+09,1.7,1.7,0.8272,0.8,1,1,0,0
2,1.293847e+09,1.9,1.9,0.8157,1.1,1,1,0,0
3,1.293851e+09,2.5,2.5,0.7831,0.8,1,1,0,0
4,1.293854e+09,2.0,2.0,0.8075,1.1,1,1,0,0


In [34]:
#train test split
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#scale the data
scaler = MinMaxScaler().fit(x_test)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)



In [35]:
#build a sequential NN
model = Sequential()
model.add(layers.Dense(128, input_dim=len(x_train[0]), activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='relu'))

model.summary()

# Define a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,  # Starting learning rate
    decay_steps=10000,            # Steps before applying decay
    decay_rate=0.96,              # Multiplicative factor
    staircase=True                # If True, learning rate decays at discrete intervals
)

#complie module
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['mean_squared_error'])





/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 128)                 │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 67,458 (263.51 KB)

 Trainable params: 67,458 (263.51 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_mean_squared_error", patience=30, mode='max')

history=model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stop])
hist = pd.DataFrame(history.history)




Epoch 1/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 37579.8359 - mean_squared_error: 37579.8359 - val_loss: 29621.2227 - val_mean_squared_error: 29621.2227
Epoch 2/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 30208.8965 - mean_squared_error: 30208.8965 - val_loss: 28123.3867 - val_mean_squared_error: 28123.3867
Epoch 3/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 29366.2461 - mean_squared_error: 29366.2461 - val_loss: 27920.6895 - val_mean_squared_error: 27920.6895
Epoch 4/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 28824.0352 - mean_squared_error: 28824.0352 - val_loss: 27321.1250 - val_mean_squared_error: 27321.1250
Epoch 5/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 28382.2520 - mean_squared_error: 28382.2520 - val_loss: 26902.2363 - val_mean_squared_error: 26902.2363
Epoch 6/100
2461/2461 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 28494.6094 - mean_squared_error: 28494.6094 - val_loss: 27405.1875 - val_mean_squared_error: 27405.1

In [37]:
pred = np.round(model.predict(x_test),1)
mse = mean_squared_error(y_test, pred)
pred


1055/1055 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


array([[611.9, 171.3],
       [320. , 223. ],
       [287. , 136.5],
       ...,
       [301.8,  31.7],
       [ 87. ,  16.3],
       [370.1,  58.8]], dtype=float32)

In [38]:
print("Root Mean Squared Error: ", mse)
r2 = r2_score(y_test, pred)
print("R2 Score: ", r2)

Root Mean Squared Error:  25553.26953125
R2 Score:  0.47888055443763733
